## 使用Transformer室内定位

In [ ]:
import copy
import torch
from torch import nn
from torch import optim
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import utils.calculate_param as cp
import dataset.data_read as data
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.metrics import mean_absolute_error, explained_variance_score, r2_score, mean_squared_error

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

### 加载数据集

In [ ]:
X_train, y_train = data.load_data('train')
X_test, y_test = data.load_data('test')

X_train.shape

In [ ]:
X_train = torch.from_numpy(X_train).float().to(device) # [len * feature]
y_train = torch.from_numpy(y_train).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device) # [len * feature]
y_test = torch.from_numpy(y_test).float().to(device)

X_train.shape

In [ ]:
dataset = TensorDataset(X_train, y_train)
data_loader = DataLoader(dataset, batch_size=100, shuffle=False)

### 定义Transformer的时序模型

In [ ]:
class RFIDTransformer(nn.Module):
    def __init__(self):
        super(RFIDTransformer, self).__init__()
        self.embedding_enc = nn.Linear(50, 128)
        self.embedding_dec = nn.Linear(2, 128)
        self.Transformer_layer = nn.Transformer(d_model=128, num_encoder_layers=3, num_decoder_layers=3, batch_first=True)
        self.FC_layer = nn.Linear(128, 2)

    def forward(self, src, tgt):
        # 使用线性层代替embedding
        src = self.embedding_enc(src).unsqueeze(0)
        tgt = self.embedding_dec(tgt).unsqueeze(0)
        out = self.Transformer_layer(src, tgt)
        out = self.FC_layer(out)
        return out


### 查看模型信息

In [ ]:
model = RFIDTransformer().to(device)
cp.get_info(model)
cp.get_parameter_number(model)

### 开始训练

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.99)
best_loss = 100000
best_epoch = 0

for epoch in range(50):
    epoch_loss = 0
    for X, y in tqdm(data_loader):  # enc_inputs : [len * feature]->[2000 * 50]
        # print(X.shape)  # [100 * 50]
        # print(y.shape)  # [100 * 50]
        outputs = model(X, y)
        outputs = outputs.squeeze()  # [100 * 2]
        # print(outputs.shape)
        loss = criterion(outputs, y)
        loss_num = loss.item()
        epoch_loss += loss_num
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        best_epoch = epoch
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(best_model_wts, './result/trans_weight.pth')

    print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(epoch_loss))

# 打印最佳的结果
print('best_loss::|',best_loss,'---best_epoch::|',best_epoch)

print('done!')

### 模型评估

In [ ]:
model = RFIDTransformer()
model.load_state_dict(torch.load('./result/trans_weight.pth'))
model.eval()
pxy = model(X_test, y_test)
pxy = pxy.cpu().detach().numpy().squeeze(0)
y_test = y_test.cpu().detach().numpy()

# 计算指标
mae = mean_absolute_error(y_test, pxy)
mse = mean_squared_error(y_test, pxy)
rmse = mse ** 0.5
evs = explained_variance_score(y_test, pxy)
r2 = r2_score(y_test, pxy)

mmax = 0
mmin = 10000
for i in range(len(pxy)):
    mmax = max(mean_absolute_error(y_test[i], pxy[i]), mmax)
    mmin = min(mean_absolute_error(y_test[i], pxy[i]), mmin)

print({'mae': mae, 'mse': mse, 'rmse': rmse, 'evs': evs, 'r2': r2, 'mmax': mmax, 'mmin': mmin})


### 绘图分析

In [ ]:
y_test = y_test[:20]
pxy = pxy[:20]
coor1 = pd.DataFrame(y_test)
coor1.columns = ['X', 'y']

coor2 = pd.DataFrame(pxy)
coor2.columns = ['PX', 'Py']

coor = pd.concat([coor1, coor2], axis=1)
coor.to_csv('./result/coordinate_all.csv')
coor

In [ ]:
plt.figure(figsize=[10, 10])
plt.xlim((0, 5))
plt.ylim((0, 5))
plt.ylabel('RFID reader')
plt.title('Coordinate Comparison')
# 画图-标准坐标
plt.scatter(y_test[:, 0], y_test[:, 1], c='black', marker='^', label='real position of RFID tag')

# 画图-预测EA坐标
plt.scatter(pxy[:, 0], pxy[:, 1], c='red', marker='x', label = 'predict position with Transformer')

plt.legend()
plt.grid('True')
plt.savefig('./result/compare_coordinate_all.jpg', dpi=750, bbox_inches = 'tight')